In [1]:
import uuid
from googleads import adwords
import sys

In [2]:
from csv import reader
import pandas as pd
import numpy as np

In [3]:
AUTH_FILE_PATH = '/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml'
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#client_2 = ad_manager.AdManagerClient.LoadFromStorage('/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml')

In [4]:
# ID of your customer here
# CUSTOMER_SERVICE_ID = '5922380045'
CUSTOMER_SERVICE_ID = '3637290511'
# Load customer account access
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)

In [84]:
# Display which field is able to query for certain report type
REPORT_TYPE = 'CAMPAIGN_PERFORMANCE_REPORT'

def get_report_fields(client, report_type):
    # Initialize appropriate service.
    report_definition_service = client.GetService(
        'ReportDefinitionService', version='v201809')

    # Get report fields.
    fields = report_definition_service.getReportFields(report_type)

    # Display results.
    print('Report type "%s" contains the following fields:' % report_type)
    for field in fields:
        print(' - %s(%s)' % (field['fieldName'], field['fieldType']))
        if 'enumValues' in field:
            print('  := [%s]' % ', '.join(field['enumValues']))

if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    get_report_fields(adwords_client, REPORT_TYPE)

Report type "CAMPAIGN_PERFORMANCE_REPORT" contains the following fields:
 - Period(BudgetPeriod)
  := []
 - UrlCustomParameters(CustomParameters)
  := []
 - AbsoluteTopImpressionPercentage(Double)
  := []
 - AccountCurrencyCode(String)
  := []
 - AccountDescriptiveName(String)
  := []
 - AccountTimeZone(String)
  := []
 - ActiveViewCpm(Money)
  := []
 - ActiveViewCtr(Double)
  := []
 - ActiveViewImpressions(Long)
  := []
 - ActiveViewMeasurability(Double)
  := []
 - ActiveViewMeasurableCost(Money)
  := []
 - ActiveViewMeasurableImpressions(Long)
  := []
 - ActiveViewViewability(Double)
  := []
 - AdNetworkType1(AdNetworkType1)
  := [UNKNOWN, SEARCH, CONTENT, YOUTUBE_SEARCH, YOUTUBE_WATCH, MIXED]
 - AdNetworkType2(AdNetworkType2)
  := [UNKNOWN, SEARCH, SEARCH_PARTNERS, CONTENT, YOUTUBE_SEARCH, YOUTUBE_WATCH, MIXED]
 - AdvertisingChannelSubType(AdvertisingChannelSubType)
  := [UNKNOWN, SEARCH_MOBILE_APP, DISPLAY_MOBILE_APP, SEARCH_EXPRESS, DISPLAY_EXPRESS, UNIVERSAL_APP_CAMPAIGN, DISPLAY

In [7]:
campaign_id = 1747836661

In [8]:
CAMPAIGN_FIELDS = ['ExternalCustomerId','CampaignId','AdvertisingChannelType', 'CampaignStatus',
                   'BiddingStrategyType','Amount','StartDate','EndDate','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
class DatePreset:
    today = 'TODAY'
    yesterday = 'YESTERDAY'
    lifetime = 'ALL_TIME'

def get_campaign_insights(client, campaign_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    report = {
        'reportName': 'CAMPAIGN_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'CAMPAIGN_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': CAMPAIGN_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
                }
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(CAMPAIGN_FIELDS)),
            columns=CAMPAIGN_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])] = df[df.columns.difference(
        ['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference(['ExternalCustomerId', 'CampaignId', 'AdvertisingChannelType', 'CampaignStatus', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions'])] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions'])].div(1000000)
    df[['StartDate','EndDate']] = df[['StartDate','EndDate']].apply( pd.to_datetime, errors='coerce' )
    return df

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    campaign_insights = get_campaign_insights(client, campaign_id=campaign_id)


In [9]:
campaign_insights.info()
campaign_insights

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 15 columns):
ExternalCustomerId        1 non-null int64
CampaignId                1 non-null int64
AdvertisingChannelType    1 non-null object
CampaignStatus            1 non-null object
BiddingStrategyType       1 non-null object
Amount                    1 non-null float64
StartDate                 1 non-null datetime64[ns]
EndDate                   1 non-null datetime64[ns]
Cost                      1 non-null float64
AverageCost               1 non-null float64
Impressions               1 non-null int64
Clicks                    1 non-null int64
Conversions               1 non-null float64
AverageCpc                1 non-null float64
CostPerConversion         1 non-null float64
dtypes: datetime64[ns](2), float64(6), int64(4), object(3)
memory usage: 200.0+ bytes


,ExternalCustomerId,CampaignId,AdvertisingChannelType,CampaignStatus,BiddingStrategyType,Amount,StartDate,EndDate,Cost,AverageCost,Impressions,Clicks,Conversions,AverageCpc,CostPerConversion
0,3637290511,1747836661,Display,enabled,cpc,100.0,2019-03-28,2019-04-27,2304.265235,3.218248,291498,716,1.0,3.218248,2304.265235


In [12]:
ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus',
                  'CpmBid','CpvBid', 'CpcBid', 'TargetCpa', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_adgroup_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_adgroup_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'ADGROUP_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'ADGROUP_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': ADGROUP_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(ADGROUP_FIELDS)),
            columns=ADGROUP_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
        ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_adgroup_insights(client)
    adgroup_insights = get_adgroup_insights(client, campaign_id=campaign_id)

In [13]:
adgroup_insights.info()
adgroup_insights

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 17 columns):
ExternalCustomerId     5 non-null int64
CampaignId             5 non-null int64
AdGroupType            5 non-null object
AdGroupId              5 non-null int64
AdGroupStatus          5 non-null object
CpmBid                 0 non-null float64
CpvBid                 0 non-null float64
CpcBid                 5 non-null float64
TargetCpa              0 non-null float64
BiddingStrategyType    5 non-null object
Cost                   5 non-null float64
AverageCost            5 non-null float64
Impressions            5 non-null float64
Clicks                 5 non-null float64
Conversions            5 non-null float64
AverageCpc             5 non-null float64
CostPerConversion      5 non-null float64
dtypes: float64(11), int64(3), object(3)
memory usage: 760.0+ bytes


,ExternalCustomerId,CampaignId,AdGroupType,AdGroupId,AdGroupStatus,CpmBid,CpvBid,CpcBid,TargetCpa,BiddingStrategyType,Cost,AverageCost,Impressions,Clicks,Conversions,AverageCpc,CostPerConversion
0,3637290511,1747836661,Display,68524904055,enabled,NaN,NaN,4.0,NaN,cpc,2051.936551,3.147142,0.251864,0.000652,1.0,3.147142,2051.936551
1,3637290511,1747836661,Display,68524904295,enabled,NaN,NaN,4.0,NaN,cpc,27.361049,3.040117,0.003910,0.000009,0.0,3.040117,0.000000
2,3637290511,1747836661,Display,68524904335,enabled,NaN,NaN,4.0,NaN,cpc,139.672192,4.108006,0.022573,0.000034,0.0,4.108006,0.000000
3,3637290511,1747836661,Display,68524904095,enabled,NaN,NaN,4.0,NaN,cpc,85.295443,4.061688,0.012319,0.000021,0.0,4.061688,0.000000
4,3637290511,1747836661,Display,68524904135,enabled,NaN,NaN,4.0,NaN,cpc,0.000000,0.000000,0.000890,0.000000,0.0,0.000000,0.000000


In [81]:
AGE_RANGE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupId', 'Id', 'AdGroupStatus',
                  'CpmBid', 'CpcBid', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# AGE_RANGE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_age_range_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_age_range_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'AGE_RANGE_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'AGE_RANGE_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': AGE_RANGE_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(AGE_RANGE_FIELDS)),
            columns=AGE_RANGE_FIELDS
        )
        df = df.append(df_temp)
#     df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
#         ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
#     df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
#         ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_age_range_insights(client)
    adgroup_insights = get_age_range_insights(client, campaign_id=1747836670, adgroup_id=68524904775)

In [78]:
AUDIENCE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupId', 'Id', 'AdGroupStatus', 'UserListName',
                  'CpmBid', 'CpcBid', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# AUDIENCE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_audience_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_audience_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'AUDIENCE_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'AUDIENCE_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': AUDIENCE_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    print(csv)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(AUDIENCE_FIELDS)),
            columns=AUDIENCE_FIELDS
        )
        df = df.append(df_temp)
#     df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
#         ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
#     df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
#         ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_audience_insights(client)
    adgroup_insights = get_audience_insights(client, campaign_id=1747836670, adgroup_id=68524904775)

In [82]:
adgroup_insights

,ExternalCustomerId,CampaignId,AdGroupId,Id,AdGroupStatus,CpmBid,CpcBid,BiddingStrategyType,Cost,AverageCost,Impressions,Clicks,Conversions,AverageCpc,CostPerConversion
0,3637290511,1747836670,68524904775,503999,enabled,--,4000000,cpc,1041389783,3165318,225969,329,3.00,3165318,347129928
1,3637290511,1747836670,68524904775,503001,enabled,--,4000000,cpc,0,0,0,0,0.00,0,0
2,3637290511,1747836670,68524904775,503004,enabled,--,4000000,cpc,469259675,3236274,90659,145,0.00,3236274,0
3,3637290511,1747836670,68524904775,503005,enabled,--,4000000,cpc,254218262,2542183,40005,100,1.00,2542183,254218262
4,3637290511,1747836670,68524904775,503003,enabled,--,4000000,cpc,166301205,3326024,36454,50,0.00,3326024,0
5,3637290511,1747836670,68524904775,503002,enabled,--,4000000,cpc,110714429,3163269,28647,35,0.00,3163269,0
6,3637290511,1747836670,68524904775,503006,enabled,--,4000000,cpc,489299123,2575259,66330,190,2.00,2575259,244649562


In [46]:
CUSTOMER_SERVICE_ID = 3637290511

In [14]:
adgroup_id = 1747836661

In [15]:
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
ad_group_service = client.GetService('AdGroupService', version='v201809')

# Construct operations and update an ad group.
operations = [{
    'operator': 'SET',
    'operand': {
        'id': ad_group_id,
        'status': 'ACTIVE'
    }
}]

if bid_micro_amount:
    operations[0]['operand']['biddingStrategyConfiguration'] = {
      'bids': [{
          'xsi_type': 'CpcBid',
          'bid': {
              'microAmount': bid_micro_amount,
          }
      }]
    }

ad_groups = ad_group_service.mutate(operations)

NameError: name 'ad_group_id' is not defined

In [35]:
from googleads import adwords
CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
AD_GROUP_ID = 68524904055
CPC_BID_MICRO_AMOUNT = '4000000'


def update_adgroup_bid(client, ad_group_id, bid_micro_amount=None):
    # Initialize appropriate service.
    ad_group_service = client.GetService('AdGroupService', version='v201809')
    
    # Construct operations and update an ad group.
    operations = [{
        'operator': 'SET',
        'operand': {
            'id': ad_group_id,
            'status': 'ENABLED'
        }
    }]
  
    if bid_micro_amount:
        operations[0]['operand']['biddingStrategyConfiguration'] = {
            'bids': [{
                'xsi_type': 'CpcBid',
                'bid': {
                    'microAmount': bid_micro_amount,
                }
            }]
        }
  
    ad_groups = ad_group_service.mutate(operations)
    
    # Display results.
    for ad_group in ad_groups['value']:
        bidding_strategy_configuration = ad_group['biddingStrategyConfiguration']
        # Find the CpcBid in the bidding strategy configuration's bids collection.
        cpc_bid_micros = None
    
        if bidding_strategy_configuration:
            bids = bidding_strategy_configuration['bids']
      
            if bids:
                for bid in bids:
                    if bid['Bids.Type'] == 'CpcBid':
                        cpc_bid_micros = bid['bid']['microAmount']
                        break
    
        print ('Ad group with name "%s", and id "%s" was updated to have status '
             '"%s" and CPC bid %d.'
             % (ad_group['name'], ad_group['id'], ad_group['status'],
                cpc_bid_micros))
    return ad_groups

# if __name__ == '__main__':
#     # Initialize client object.
#     adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adwords_client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
#     update_adgroup_bid(adwords_client, AD_GROUP_ID, CPC_BID_MICRO_AMOUNT)

In [37]:
CUSTOMER_SERVICE_ID = '3207327801'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
CAMPAIGN_ID = 1745022368
AD_GROUP_ID = 74005199208
CPC_BID_MICRO_AMOUNT = '8000000'
adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
adwords_client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
update_adgroup_bid(adwords_client, AD_GROUP_ID, CPC_BID_MICRO_AMOUNT)

Ad group with name "[寶寶]BT_1904_F25-44/育兒教育/嬰兒推車/嬰幼兒服飾", and id "74005199208" was updated to have status "ENABLED" and CPC bid 8000000.


{
    'ListReturnValue.Type': 'AdGroupReturnValue',
    'value': [
        {
            'id': 74005199208,
            'campaignId': 1745022368,
            'campaignName': '[寶寶]巧連智_GDN_BT_嬰幼兒',
            'name': '[寶寶]BT_1904_F25-44/育兒教育/嬰兒推車/嬰幼兒服飾',
            'status': 'ENABLED',
            'settings': [
                {
                    'Setting.Type': 'TargetingSetting',
                    'details': [
                        {
                            'criterionTypeGroup': 'USER_INTEREST_AND_LIST',
                            'targetAll': False
                        },
                        {
                            'criterionTypeGroup': 'VERTICAL',
                            'targetAll': True
                        },
                        {
                            'criterionTypeGroup': 'GENDER',
                            'targetAll': False
                        },
                        {
                            'criterionTypeGroup': 'AGE_RA

In [65]:
# CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
ad_group_service = client.GetService('AdGroupService', version='v201809')

# Construct operations and update an ad group.

operations = [{
    'operator': 'SET',
    'operand': {
        'id': 68524904775,
        'status': 'ENABLED'
    }
}]
# ad_groups = ad_group_service.get(selector)
ad_groups = ad_group_service.mutate(operations)

In [66]:
ad_groups

{
    'ListReturnValue.Type': 'AdGroupReturnValue',
    'value': [
        {
            'id': 68524904775,
            'campaignId': 1747836670,
            'campaignName': '華友旅行_4月_CPC$8_3/28-4/27_PT',
            'name': 'PT-25-54_All',
            'status': 'ENABLED',
            'settings': [
                {
                    'Setting.Type': 'TargetingSetting',
                    'details': [
                        {
                            'criterionTypeGroup': 'USER_INTEREST_AND_LIST',
                            'targetAll': False
                        },
                        {
                            'criterionTypeGroup': 'VERTICAL',
                            'targetAll': False
                        },
                        {
                            'criterionTypeGroup': 'GENDER',
                            'targetAll': False
                        },
                        {
                            'criterionTypeGroup': 'AGE_RANGE',
       

In [63]:
# CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
campaign_service = client.GetService('CampaignService', version='v201809')

# Construct operations and update an ad group.

operations = [{
    'operator': 'SET',
    'operand': {
        'id': 1747836670,
        'status': 'ENABLED'
    }
}]
# ad_groups = ad_group_service.get(selector)
campaigns = campaign_service.mutate(operations)

In [64]:
campaigns

{
    'ListReturnValue.Type': 'CampaignReturnValue',
    'value': [
        {
            'id': 1747836670,
            'campaignGroupId': None,
            'name': '華友旅行_4月_CPC$8_3/28-4/27_PT',
            'status': 'ENABLED',
            'servingStatus': 'SERVING',
            'startDate': '20190328',
            'endDate': '20190427',
            'budget': {
                'budgetId': 1806201010,
                'name': '華友旅行_4月_CPC$8_3/28-4/27_PT',
                'amount': {
                    'ComparableValue.Type': 'Money',
                    'microAmount': 100000000
                },
                'deliveryMethod': 'STANDARD',
                'referenceCount': None,
                'isExplicitlyShared': False,
                'status': 'ENABLED'
            },
            'conversionOptimizerEligibility': {
                'eligible': False,
                'rejectionReasons': [
                    'NOT_ENOUGH_CONVERSIONS'
                ]
            },
            'adS

In [61]:
def main(client):
    # Initialize appropriate service.
    constant_data_service = client.GetService(
        'ConstantDataService', version='v201809')
  
    # Get all ageranges.
    ageranges = constant_data_service.getAgeRangeCriterion()
    print(ageranges)
    # Display results.
    for agerange in ageranges:
        print ('agerange with name "%s" and ID "%s" was found.'
               % (agerange['ageRangeType'], agerange['id']))
  
    # Get all carriers.
    carriers = constant_data_service.getCarrierCriterion()
  
    # Display results.
    for carrier in carriers:
        print ('Carrier with name "%s", ID "%s", and country code "%s" was '
               'found.' % (
                   carrier['name'], carrier['id'],
                   getattr(carrier, 'countryCode', 'N/A')))


if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
  
    main(adwords_client)

[{
    'id': 503999,
    'type': None,
    'Criterion.Type': 'AgeRange',
    'ageRangeType': 'AGE_RANGE_UNDETERMINED'
}, {
    'id': 503006,
    'type': None,
    'Criterion.Type': 'AgeRange',
    'ageRangeType': 'AGE_RANGE_65_UP'
}, {
    'id': 503005,
    'type': None,
    'Criterion.Type': 'AgeRange',
    'ageRangeType': 'AGE_RANGE_55_64'
}, {
    'id': 503004,
    'type': None,
    'Criterion.Type': 'AgeRange',
    'ageRangeType': 'AGE_RANGE_45_54'
}, {
    'id': 503003,
    'type': None,
    'Criterion.Type': 'AgeRange',
    'ageRangeType': 'AGE_RANGE_35_44'
}, {
    'id': 503002,
    'type': None,
    'Criterion.Type': 'AgeRange',
    'ageRangeType': 'AGE_RANGE_25_34'
}, {
    'id': 503001,
    'type': None,
    'Criterion.Type': 'AgeRange',
    'ageRangeType': 'AGE_RANGE_18_24'
}, {
    'id': 503000,
    'type': None,
    'Criterion.Type': 'AgeRange',
    'ageRangeType': None
}]
agerange with name "AGE_RANGE_UNDETERMINED" and ID "503999" was found.
agerange with name "AGE_RANGE_